<a href="https://colab.research.google.com/github/Rajadurai97/stock-price-prediction/blob/main/stock_price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import quandl
import pandas as pd
import numpy as np
import math
from sklearn import preprocessing, svm
from sklearn.linear_model import LinearRegression

In [ ]:
quandl.ApiConfig.api_key = "ZBW1ZvS8-UhrSYcxaCRf"

In [ ]:
df = quandl.get("EOD/AAPL")

In [ ]:
df.head(10)

,Open,High,Low,Close,Volume,Dividend,Split,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Volume
Date,,,,,,,,,,,,
2013-09-03,493.1000,500.6000,487.35,488.580,11854600.0,0.0,1.0,15.619650,15.857223,15.437510,15.476472,331928800.0
2013-09-04,499.5600,502.2400,496.28,498.691,12322600.0,0.0,1.0,15.824280,15.909173,15.720381,15.796753,345032800.0
2013-09-05,500.2500,500.6800,493.64,495.270,8441700.0,0.0,1.0,15.846136,15.859757,15.636755,15.688388,236367600.0
2013-09-06,498.4400,499.3800,489.95,498.220,12840200.0,0.0,1.0,15.788802,15.818578,15.519869,15.781833,359525600.0
2013-09-09,505.0000,507.9200,503.48,506.170,12167400.0,0.0,1.0,15.996600,16.089095,15.948451,16.033661,340687200.0
2013-09-10,506.2000,507.4500,489.50,494.640,26542700.0,0.0,1.0,16.034611,16.074207,15.505615,15.668432,743195600.0
2013-09-11,467.0101,473.6900,464.81,467.710,32096300.0,0.0,1.0,14.793215,15.004810,14.723524,14.815385,898696400.0
2013-09-12,468.5000,475.4000,466.01,472.690,14430400.0,0.0,1.0,14.840410,15.058977,14.761535,14.973134,404051200.0
2013-09-13,469.3402,471.8292,464.70,464.900,10672700.0,0.0,1.0,14.867024,14.945867,14.720039,14.726374,298835600.0


In [ ]:
df.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Dividend', 'Split',
       'Adj_Open', 'Adj_High', 'Adj_Low', 'Adj_Close', 'Adj_Volume'],
      dtype='object')

In [ ]:
df.shape

(1090, 12)

In [ ]:
df.isnull().sum()

Open          0
High          0
Low           0
Close         0
Volume        0
Dividend      0
Split         0
Adj_Open      0
Adj_High      0
Adj_Low       0
Adj_Close     0
Adj_Volume    0
dtype: int64

**Data manipulation & Feature engineering**

In [ ]:
df['HL_PCT'] = df['Adj_High'] - df['Adj_Low'] / df['Adj_Close'] *100.0 #High low percentage
df['PCT_change'] = df['Adj_Close'] - df['Adj_Open'] / df['Adj_Open'] *100.0 #percentage change

In [ ]:
df.head(10)

,Open,High,Low,Close,Volume,Dividend,Split,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Volume,HL_PCT,PCT_change
Date,,,,,,,,,,,,,,
2013-09-03,493.1000,500.6000,487.35,488.580,11854600.0,0.0,1.0,15.619650,15.857223,15.437510,15.476472,331928800.0,-83.891027,-84.523528
2013-09-04,499.5600,502.2400,496.28,498.691,12322600.0,0.0,1.0,15.824280,15.909173,15.720381,15.796753,345032800.0,-83.607362,-84.203247
2013-09-05,500.2500,500.6800,493.64,495.270,8441700.0,0.0,1.0,15.846136,15.859757,15.636755,15.688388,236367600.0,-83.811129,-84.311612
2013-09-06,498.4400,499.3800,489.95,498.220,12840200.0,0.0,1.0,15.788802,15.818578,15.519869,15.781833,359525600.0,-82.521513,-84.218167
2013-09-09,505.0000,507.9200,503.48,506.170,12167400.0,0.0,1.0,15.996600,16.089095,15.948451,16.033661,340687200.0,-83.379463,-83.966339
2013-09-10,506.2000,507.4500,489.50,494.640,26542700.0,0.0,1.0,16.034611,16.074207,15.505615,15.668432,743195600.0,-82.886654,-84.331568
2013-09-11,467.0101,473.6900,464.81,467.710,32096300.0,0.0,1.0,14.793215,15.004810,14.723524,14.815385,898696400.0,-84.375147,-85.184615
2013-09-12,468.5000,475.4000,466.01,472.690,14430400.0,0.0,1.0,14.840410,15.058977,14.761535,14.973134,404051200.0,-83.527835,-85.026866
2013-09-13,469.3402,471.8292,464.70,464.900,10672700.0,0.0,1.0,14.867024,14.945867,14.720039,14.726374,298835600.0,-85.011113,-85.273626


In [ ]:
df2 = df[['Adj_Close','PCT_change','HL_PCT','Adj_Volume']]

In [ ]:
df2.head()

,Adj_Close,PCT_change,HL_PCT,Adj_Volume
Date,,,,
2013-09-03,15.476472,-84.523528,-83.891027,331928800.0
2013-09-04,15.796753,-84.203247,-83.607362,345032800.0
2013-09-05,15.688388,-84.311612,-83.811129,236367600.0
2013-09-06,15.781833,-84.218167,-82.521513,359525600.0
2013-09-09,16.033661,-83.966339,-83.379463,340687200.0


In [ ]:
forecast_col = 'Adj_Close'
df2.fillna(value=-99999, inplace=True)
forecast_out = int(math.ceil(0.01 * len(df2)))

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [ ]:
df2['label'] = df2[forecast_col].shift(-forecast_out)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df2.head(10)

,Adj_Close,PCT_change,HL_PCT,Adj_Volume,label
Date,,,,,
2013-09-03,15.476472,-84.523528,-83.891027,331928800.0,14.719406
2013-09-04,15.796753,-84.203247,-83.607362,345032800.0,14.960780
2013-09-05,15.688388,-84.311612,-83.811129,236367600.0,14.805882
2013-09-06,15.781833,-84.218167,-82.521513,359525600.0,15.541726
2013-09-09,16.033661,-83.966339,-83.379463,340687200.0,15.492944
2013-09-10,15.668432,-84.331568,-82.886654,743195600.0,15.253154
2013-09-11,14.815385,-85.184615,-84.375147,898696400.0,15.401716
2013-09-12,14.973134,-85.026866,-83.527835,404051200.0,15.291799
2013-09-13,14.726374,-85.273626,-85.011113,298835600.0,15.101740


In [ ]:
df2.dropna(inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


**Scale the data and fit to the classifier**

In [ ]:
# matrix of features
x = np.array(df2.drop(['label'], 1)) #independant var
y = np.array(df2['label']) #dependant var

In [ ]:
# Scaling the features
x = preprocessing.scale(x)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
#fitting linear regression to the training set
from sklearn.linear_model import LinearRegression
classifier = LinearRegression()
classifier.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
confidence_score = classifier.score(x_test, y_test)
confidence_score

0.9598480584217722

In [ ]:
print(f'The Intercept is :\n {classifier.intercept_}')
feature_columns = ['Adj_Close','PCT_change','HL_PCT','Adj_Volume']
a = zip(feature_columns, classifier.coef_)
print(f'The Co_efficient is :\n {tuple(a)}')


The Intercept is :
 47.158728981894754
The Co_efficient is :
 (('Adj_Close', 0.5247009867283015), ('PCT_change', 0.5247009894250081), ('HL_PCT', -0.06057311756700411), ('Adj_Volume', -5.699192229258188e-10))


In [ ]:
# Predicting the Test set results
y_pred = classifier.predict(x_test)

**Evaluate the model using the accuracy matrix**

In [ ]:
from sklearn.metrics import mean_squared_error,r2_score

# The mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(y_test, y_pred))


Mean squared error: 1.38
Coefficient of determination: 0.96
